In [27]:
import pandas as pd
import numpy as np
import annoy
#import faiss
import torch

from scipy import sparse
from implicit.als import AlternatingLeastSquares
from implicit.approximate_als import AnnoyAlternatingLeastSquares, NMSLibAlternatingLeastSquares

# from itertools import combinations
# from torch_geometric.nn import models

In [2]:
data = pd.read_csv('v3.inter', lineterminator='\n')
data = data[['item_id:token', 'user_id:token', 'timestamp:float']]
data.columns = ['item_id', 'user_id', 'time']
data = data.sort_values('item_id').reset_index(drop=True)
data.head()

item2vec = {v:i for i, v in enumerate(sorted(data['item_id'].unique()))}
user2vec = {v:i for i, v in enumerate(sorted(data['user_id'].unique()))}
vec2item = {i:v for i, v in enumerate(sorted(data['item_id'].unique()))}
vec2user = {i:v for i, v in enumerate(sorted(data['user_id'].unique()))}
data['item_id'] = data['item_id'].apply(lambda x : item2vec[x])
data['user_id'] = data['user_id'].apply(lambda x : user2vec[x])

data['purchased'] = 1
matrix = sparse.csr_matrix((data['purchased'], (data['user_id'], data['item_id'])))
matrix



<645103x18467 sparse matrix of type '<class 'numpy.int64'>'
	with 8800191 stored elements in Compressed Sparse Row format>

In [3]:
model = AnnoyAlternatingLeastSquares()
model.fit(matrix)

c:\Users\user\AppData\Local\Programs\Python\Python36\lib\site-packages\implicit\utils.py:29: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  "OpenBLAS detected. Its highly recommend to set the environment variable "
100%|██████████| 15/15 [10:15<00:00, 41.03s/it]


In [142]:
model.n_trees

50

In [22]:
users = model.model.user_factors
users.shape

(645103, 100)

In [124]:
an = annoy.AnnoyIndex(users.shape[1], 'angular')
for i in range(users.shape[0]) :
    an.add_item(i, users[i])
an.build(20)

True

In [144]:
an.save('annoy.ann')

True

In [126]:
result = an.get_nns_by_item(0, 10)
result

[0, 60009, 46237, 235529, 115560, 92575, 116176, 265277, 103094, 71804]

In [127]:
result2 = [[vec2user[j], data[data.user_id == j]['item_id'].values] for j in result]
result2

[[11, array([   6,  101,  110,  117,  217,  242,  250,  298, 5742, 6091, 6121,
         6905, 6926, 7500, 8180, 8234, 8290, 8312, 8313, 8438], dtype=int64)],
 [1418414, array([ 101,  110,  425, 4085, 5999, 6689, 6694, 6902, 6905, 6926],
        dtype=int64)],
 [1238006,
  array([   46,   110,   235,   352,   405,  1894,  5216,  5685,  6121,
          6574,  6905,  6926,  8481, 14864], dtype=int64)],
 [4544771,
  array([ 110, 1931, 5796, 6539, 6904, 6905, 6926, 7341, 8169, 8717, 9384],
        dtype=int64)],
 [2319449,
  array([ 110,  217, 4704, 5690, 6902, 6904, 6905, 9320], dtype=int64)],
 [1915834, array([  43,  110,  502, 5430, 6902, 6904, 6905, 6926, 7742, 8151],
        dtype=int64)],
 [2330245,
  array([ 110,  401,  645, 5483, 6529, 6902, 6904, 6905], dtype=int64)],
 [5067595, array([  110,  1514,  6902,  6905,  6926,  7654,  8325, 13394],
        dtype=int64)],
 [2095130,
  array([   41,   209,   400,   405,   413,   414,   777,   867,  1954,
          2313,  5586,  5636,  5742,

In [35]:
[[vec2user[j] for j in result]]

[11, 141262, 14065, 14948, 107478, 129335, 156297, 67696, 156152, 41788]

In [36]:
data.head(2)

,item_id,user_id,time,purchased
0,0,95450,1.544486e+09,1
1,0,130922,1.573603e+09,1


In [38]:
tmp = data.groupby(by=['user_id'], as_index=False)['item_id'].apply(list)

In [54]:
tmp.loc[tmp['user_id'].isin(result)]

,user_id,item_id
0,0,"[6, 101, 110, 117, 217, 242, 250, 298, 5742, 6..."
77,77,"[113, 5742, 6134, 6136, 6269]"
81,81,"[1170, 1411, 3254, 3866, 5742, 5752, 5826, 619..."
229,229,"[352, 569, 891, 5742, 5829, 7655, 9870]"
381,381,"[39, 110, 308, 392, 425, 457, 834, 852, 2067, ..."
645,645,"[0, 10, 82, 115, 516, 5748, 6817, 8072, 8717]"
784,784,"[1026, 4045, 4223, 4305, 6083, 6904, 6905, 987..."
870,870,"[113, 802, 5742, 5864, 13367]"
983,983,"[23, 173, 288, 544, 5130, 5748, 8378, 13329, 1..."
986,986,"[2957, 3903, 5430, 6904, 6905, 6960, 15848]"


In [59]:
prod_si = pd.read_csv('prod_sideinfo.csv')
prod_si.head()

,item_id,title,brand,original_price,selling_price,wish_count,share_count,scrap_count,qa_count,review_count,...,is_selling,is_buyable,category0,category1,delivery_fee,delivery_fee_free_threshold,img_main,img_sub_list,option_names,max_option_price
0,265527,마르틴 원목 사이드보드 2colors,장미맨숀,938000,469000,9380,517,9380,245,198,...,True,True,가구,거실장·TV장,30000,0,https://image.ohou.se/image/central_crop/bucke...,['https://image.ohou.se/image/central_crop/buc...,"['색상:클래식화이트', '색상:램프블랙']",469000
1,456986,600ml 스텐 우유 스팀피쳐/눈금형,벨라쿠진,19800,9900,1118,31,1118,10,153,...,True,True,주방용품,커피·티용품,3000,10000,https://image.ohou.se/image/central_crop/bucke...,['https://image.ohou.se/image/central_crop/buc...,['선택:600ml 스텐 우유 스팀피쳐/눈금형'],9900
2,118895,PETALO CHAIR,잭슨카멜레온,0,0,3375,108,3375,0,4,...,False,True,가구,의자,0,0,https://image.ohou.se/image/central_crop/bucke...,[],[],0
3,312929,스틸 접이식 실내화걸이 3단,네이쳐리빙,13000,8900,5458,308,5458,114,4968,...,True,True,수납·정리,현관·신발정리,0,0,https://image.ohou.se/image/central_crop/bucke...,['https://image.ohou.se/image/central_crop/buc...,"['색상:화이트', '색상:블랙']",8900
4,232064,"샤워볼 20g, 40g, 60g",소소모소,4500,1800,638,10,638,8,206,...,True,True,생활용품,욕실용품,3000,50000,https://image.ohou.se/image/central_crop/bucke...,['https://image.ohou.se/image/central_crop/buc...,"['중량:미니샤워볼 20g', '중량:베이직샤워볼 40g', '중량:빅샤워볼 60g']",2800


In [128]:
d = []
for user, items in result2 :
    for i in items :
        d.append([user, prod_si[prod_si.item_id == vec2item[i]].title.item(), prod_si[prod_si.item_id == vec2item[i]].category1.item()])
tmpp = pd.DataFrame(d, columns=['user_id', 'title', 'category'])

In [129]:
tmpp

,user_id,title,category
0,11,미카 스탠드 2colors,단스탠드
1,11,마리에 1500 스탠드 전신거울 4colors,거울
2,11,[오늘의딜/10%쿠폰] 냉기차단 워셔블 모던 러그 카페트 5size 8color,러그·카페트
3,11,리뉴얼 접이식 릴렉스체어/안락의자 2colors,의자
4,11,국내생산 고급콘주화이바 구름이불솜(SS/Q),이불·이불솜
5,11,물튀방지! 국내생산 실리콘 싱크대 물막이 3colors,주방잡화
6,11,[5%쿠폰]커튼링 2colors,커튼·부자재
7,11,[5%쿠폰][캔들랜덤증정]New SAMONY 캔들워머 버튼 6colors (할로겐전...,디퓨저·캔들
8,11,[10%쿠폰] (기간한정)네이쳐 양모 먼지털이개_3size,청소용품
9,11,로켓 미니스투키_2colors,플라워·식물


In [130]:
tmpp['user_id'].unique()

array([     11, 1418414, 1238006, 4544771, 2319449, 1915834, 2330245,
       5067595, 2095130, 1573507], dtype=int64)

In [134]:
tmpp.loc[tmpp['user_id']==11, 'category'].value_counts()

커튼·부자재       4
이불·이불솜       2
홈갤러리         2
플라워·식물       2
베개·베개커버      1
단스탠드         1
주방잡화         1
디퓨저·캔들       1
의자           1
청소용품         1
테이블·식탁·책상    1
거울           1
러그·카페트       1
시계           1
Name: category, dtype: int64

In [132]:
tmpp.loc[tmpp['user_id']==1418414, 'category'].value_counts()

행거·옷장        2
주방수납·정리      1
화장대·테이블정리    1
이불·이불솜       1
거울           1
화장대·콘솔       1
토퍼·패드        1
러그·카페트       1
베개·베개커버      1
Name: category, dtype: int64

In [133]:
tmpp.loc[tmpp['user_id']==1238006, 'category'].value_counts()

의자           2
데스크스탠드       1
베개·베개커버      1
커튼·부자재       1
월데코·장식       1
디퓨저·캔들       1
선반           1
이불·이불솜       1
침대           1
테이블·식탁·책상    1
서랍·수납장       1
러그·카페트       1
시계           1
Name: category, dtype: int64

In [143]:
import pickle
with open('vec2user.pickle', 'wb') as f :
    pickle.dump(vec2user, f, pickle.HIGHEST_PROTOCOL)
with open('vec2item.pickle', 'wb') as f :
    pickle.dump(vec2item, f, pickle.HIGHEST_PROTOCOL)

# 모델 로드

In [145]:
ann = annoy.AnnoyIndex(100, 'angular')
ann.load('annoy.ann')

True

In [146]:
ann.get_nns_by_item(0, n=10)

[0, 60009, 46237, 235529, 115560, 92575, 116176, 265277, 103094, 71804]

In [147]:
with open('vec2user.pickle', 'rb') as f :
    vec2user = pickle.load(f)
with open('vec2item.pickle', 'rb') as f :
    vec2item = pickle.load(f)
vec2user

{0: 11,
 1: 22,
 2: 27,
 3: 332,
 4: 417,
 5: 793,
 6: 835,
 7: 914,
 8: 962,
 9: 1126,
 10: 1664,
 11: 1720,
 12: 1792,
 13: 2018,
 14: 2048,
 15: 2114,
 16: 2120,
 17: 2147,
 18: 2368,
 19: 2388,
 20: 2615,
 21: 3007,
 22: 3034,
 23: 3166,
 24: 3258,
 25: 3312,
 26: 3765,
 27: 3911,
 28: 4133,
 29: 4476,
 30: 4523,
 31: 4527,
 32: 4666,
 33: 4973,
 34: 5140,
 35: 5435,
 36: 5504,
 37: 5507,
 38: 5511,
 39: 5830,
 40: 6061,
 41: 6355,
 42: 6494,
 43: 6607,
 44: 6670,
 45: 6736,
 46: 7018,
 47: 7194,
 48: 7416,
 49: 7747,
 50: 7853,
 51: 7947,
 52: 8544,
 53: 8642,
 54: 8754,
 55: 8940,
 56: 8959,
 57: 8984,
 58: 9197,
 59: 9742,
 60: 9913,
 61: 10160,
 62: 10198,
 63: 10240,
 64: 10273,
 65: 11536,
 66: 11986,
 67: 12082,
 68: 12207,
 69: 12569,
 70: 12597,
 71: 12599,
 72: 12887,
 73: 13402,
 74: 13666,
 75: 13751,
 76: 13937,
 77: 14065,
 78: 14153,
 79: 14735,
 80: 14821,
 81: 14948,
 82: 14961,
 83: 15171,
 84: 15178,
 85: 15308,
 86: 15532,
 87: 15598,
 88: 15669,
 89: 15886,
 90